In [3]:
%pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install numpy as np

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


In [5]:
import tensorflow as tf
from tensorflow import keras

In [6]:
%pip install opencv-python-headless

Note: you may need to restart the kernel to use updated packages.


In [7]:
import cv2
import os
import numpy as np

In [37]:
project_path=r"C:\Users\Asus\Desktop\NeuroDoku"
sudoku_images_digits=os.path.join(project_path,r"digits updated\digits updated")


In [38]:
sudoku_images_digits

'C:\\Users\\Asus\\Desktop\\NeuroDoku\\digits updated\\digits updated'

In [40]:
data_list=os.listdir(sudoku_images_digits)
print(data_list)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [41]:
data_list

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [14]:
prefix="image"
suffix=".jpg"
pre_len=len(prefix)
suff_len=len(suffix)

In [15]:
data_X=[]
data_Y=[]

In [16]:
print(type(data_list))

<class 'list'>


In [17]:
count=1
for image_name in data_list:
    if image_name.lower().endswith(".jpg"):
        print(count)
        image_path=os.path.join(data_list_path,image_name)
        pic=cv2.imread(image_path)
        pic=cv2.resize(pic,(32,32))
        data_X.append(pic)
        label=int(image_name[pre_len:-suff_len])
        data_Y.append(label)
        count+=1




1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200


In [18]:
len(data_Y)

200

In [19]:
if len(data_X)==len(data_Y):
    print("Toatl Data Points :", len(data_Y))

Toatl Data Points : 200


In [20]:
data_X=np.array(data_X)
data_Y=np.array(data_Y)

In [21]:
from tensorflow.keras.utils import to_categorical

In [22]:
train_y_categorical=to_categorical(data_Y,np.max(data_Y)+1)

In [23]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [24]:
import sklearn

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
train_X, test_X, train_y, test_y = train_test_split(data_X,data_Y,test_size=0.05)
train_X, valid_X, train_y, valid_y = train_test_split(train_X,train_y,test_size=0.2)
print("Training Set Shape = ",train_X.shape)
print("Validation Set Shape = ",valid_X.shape)
print("Test Set Shape = ",test_X.shape)

Training Set Shape =  (152, 32, 32, 3)
Validation Set Shape =  (38, 32, 32, 3)
Test Set Shape =  (10, 32, 32, 3)


In [27]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

In [28]:
def Prep(img):
    if len(img.shape) > 2 and img.shape[2] > 1:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) # convert to grayscale
    img = img.astype(np.uint8) # convert to 8-bit unsigned integer
    img = cv2.equalizeHist(img) # histogram equalization
    img = img/255 # normalizing
    return img

train_X = np.array(list(map(Prep, train_X)))
test_X = np.array(list(map(Prep, test_X)))
valid_X= np.array(list(map(Prep, valid_X)))

#Reshaping the images
train_X = train_X.reshape(train_X.shape[0], train_X.shape[1], train_X.shape[2],1)
test_X = test_X.reshape(test_X.shape[0], test_X.shape[1], test_X.shape[2],1)
valid_X = valid_X.reshape(valid_X.shape[0], valid_X.shape[1], valid_X.shape[2],1)

#Augmentation
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.2, shear_range=0.1, rotation_range=10)
datagen.fit(train_X)

In [29]:
data_classes = np.max(data_Y) + 1
train_y = to_categorical(train_y, data_classes)
test_y = to_categorical(test_y, data_classes)
valid_y = to_categorical(valid_y, data_classes)

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D


In [31]:
model = Sequential()

model.add((Conv2D(60,(5,5),input_shape=(32, 32, 1) ,padding = 'Same' ,activation='relu')))
model.add((Conv2D(60, (5,5),padding="same",activation='relu')))
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.25))

model.add((Conv2D(30, (3,3),padding="same", activation='relu')))
model.add((Conv2D(30, (3,3), padding="same", activation='relu')))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(500,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.summary()

c:\Users\Asus\anaconda3\envs\sudoku_solver\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 60)     │         1,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 60)     │        90,060 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 60)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 30)     │        16,230 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 30)     │         8,130 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 30)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 8, 8, 30)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1920)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 500)            │       960,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 500)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,081,490 (4.13 MB)

 Trainable params: 1,081,490 (4.13 MB)

 Non-trainable params: 0 (0.00 B)

In [32]:
from tensorflow.keras.optimizers import RMSprop

In [35]:
train_y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [34]:
optimizer = RMSprop(learning_rate=0.001, rho=0.9, epsilon = 1e-08, decay=0.0)
model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

train_y_int = np.argmax(train_y, axis=1)
valid_y_int = np.argmax(valid_y, axis=1)

history = model.fit(train_X, train_y_int, 
                    epochs=30, 
                    validation_data=(valid_X, valid_y_int), 
                    verbose=2, 
                    batch_size=32)

Epoch 1/30


InvalidArgumentError: Graph execution error:

Detected at node compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "c:\Users\Asus\anaconda3\envs\sudoku_solver\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "c:\Users\Asus\anaconda3\envs\sudoku_solver\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "c:\Users\Asus\anaconda3\envs\sudoku_solver\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "c:\Users\Asus\anaconda3\envs\sudoku_solver\Lib\site-packages\tornado\platform\asyncio.py", line 211, in start

  File "c:\Users\Asus\anaconda3\envs\sudoku_solver\Lib\asyncio\base_events.py", line 645, in run_forever

  File "c:\Users\Asus\anaconda3\envs\sudoku_solver\Lib\asyncio\base_events.py", line 1999